In [1]:
import ipyparallel as ipp
c = ipp.Client(profile='remote')#, cluster_id='remote')
dview = c[:]
print(c.ids)
%px from mpi4py import MPI
%px import numpy as np
%px comm = MPI.COMM_WORLD

[0, 1]


In [27]:
dview.run('D://code/workplace_py/Parallel/PythonCode/rank_mpi.py')
%px totalhost = hostname()
dview['totalhost']

['LAPTOP-NBMNPREL', 'LENOVO']

In [2]:
with c[:].sync_imports():
    import numpy as np
    from mpi4py import MPI

importing numpy on engine(s)
importing MPI from mpi4py on engine(s)


In [3]:
%%px
print(comm.rank)

[stdout:0] 0
[stdout:1] 0


In [4]:
dview = c[:]
dview.scatter('a',np.arange(10,dtype='float'))
dview['a']

[array([0., 1., 2., 3., 4.]), array([5., 6., 7., 8., 9.])]

In [ ]:
#%%px
#if comm.rank == 0:
c = np.arange(100, dtype='float64')
#else:
#    c = None

In [ ]:
dview['c']

In [5]:
%%px
recv_data = np.empty(5, dtype='float64')
comm.Scatter(a, recv_data, root=0)

In [6]:
%%px
print(recv_data)

[stdout:0] [0. 1. 2. 3. 4.]
[stdout:1] [5. 6. 7. 8. 9.]


In [10]:
%%px
def psum(c):
    locsum = np.sum(c)
    rcvBuf = np.array(0.0,'float64')
    MPI.COMM_WORLD.Allreduce([locsum/10, MPI.DOUBLE],
        [rcvBuf, MPI.DOUBLE],
        op=MPI.SUM)
    return rcvBuf

In [11]:
%px respective_mu = psum(recv_data)
dview['respective_mu']
#each_mu = dview['respective_mu'][0]+dview['respective_mu'][1]
#print(each_mu)

[array(1.), array(3.5)]

In [12]:
%%px
def local_var_sum_arg(mu):
    l_sum = np.array(0.0, 'float64')
    for i in recv_data:
        l_sum += (i - mu)**2
    return l_sum

In [13]:
%px lcv = local_var_sum_arg(each_mu)
dview['lcv']

[array(41.25), array(41.25)]

In [14]:
%%px
def g_var_sum(lcv):
    rec = np.array(0.0,'float64')
    total_sum = comm.reduce(lcv, op=MPI.SUM)
    return total_sum

In [23]:
%px sum = g_var_sum(lcv)
dview["sum"]
result_sum = dview["sum"][0]+dview["sum"][1]
print(result_sum)

82.5


In [24]:
#%%px
#if comm.rank ==0:
print("variance = ",result_sum/10)

variance =  8.25
